# Process rankings data from csv files and format into ML readable data

In [1]:
import pandas as pd

In [2]:
bz = pd.read_csv('banzuke_2020.csv')
ht = pd.read_csv('hoshitori_2020.csv')

In [3]:
bz.head()

,rank,name,pob,stable,birthdate,height,weight,year,month
0,Y1e,Hakuho,Mongolia,Miyagino,11.03.1985,193.0,150.7,2020,1
1,Y1w,Kakuryu,Mongolia,Michinoku,10.08.1985,186.0,150.0,2020,1
2,O1e,Takakeisho,Hyogo,Chiganoura,05.08.1996,173.0,149.0,2020,1
3,O1w,Goeido,Osaka,Sakaigawa,06.04.1986,183.0,158.2,2020,1
4,S1e,Asanoyama,Toyama,Takasago,01.03.1994,189.0,158.0,2020,1


In [42]:
bz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rank       350 non-null    object 
 1   name       350 non-null    object 
 2   pob        350 non-null    object 
 3   stable     350 non-null    object 
 4   birthdate  350 non-null    object 
 5   height     350 non-null    float64
 6   weight     350 non-null    float64
 7   year       350 non-null    int64  
 8   month      350 non-null    int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 24.7+ KB


In [4]:
ht.head()

,Result,East,Rank,West,Result.1,year,month
0,1-3-11,Hakuho,Y,Kakuryu,1-4-10,2020,1
1,11-4,Takakeisho,O,Goeido,5-10,2020,1
2,10-5,Asanoyama,S,Takayasu,6-9 ↓,2020,1
3,5-10 ↓,Abi,K,Daieisho,7-8 ↓,2020,1
4,9-6 S ↑,Endo,M1,Myogiryu,5-10,2020,1


In [19]:
ht.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Result    478 non-null    object
 1   East      478 non-null    object
 2   Rank      478 non-null    object
 3   West      472 non-null    object
 4   Result.1  472 non-null    object
 5   year      478 non-null    int64 
 6   month     478 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 26.3+ KB


# the ideal format for ML processing will have the following structure

Each observation will represent one wrestler, tournament rank, tournament results, next tournament rank
- additional information might be included such as height / weight at beginning of each tournament

## existing problems
- rank is not normalized between bz and ht (uneeded - just need results for x tournament)
- ht is not flattned and has two wrestlers per row
- results are not normalized
- results have strange characters (which are embedded with extra info - but will ignore for now)

## Flatten ht

#### process
- take ht
- slice for east (keep rank, year, month, result)
- slice for west (keep rank, year, month ,result)
- rename columns
- join to bz on name, year, month

In [18]:
west_cols = ['Result', 'West', 'Rank', 'year','month']
east_cols = ['Result.1', 'East', 'Rank', 'year', 'month']
west = ht.loc[:,west_cols]
east = ht.loc[:,east_cols]
east.rename(columns = {'East': 'name', 'Result.1': 'Result'}, inplace = True)
west.rename(columns = {'West': 'name'}, inplace = True)
ht_new = pd.concat([east,west], ignore_index = True)
ht_new.shape

(956, 5)

#### normalize results
- remove speical characters 
- and split win-loss-intai

In [24]:
ht_new.Result.str.split('-', expand = True)

,0,1,2
0,1,4,10
1,5,10,None
2,6,9 ↓,None
3,7,8 ↓,None
4,5,10,None
...,...,...,...
951,3,4,None
952,1,6,None
953,2,5,None
954,5,2,None


In [35]:
ht_new.Result.str.split(expand = True)[[0]]

,0
0,1-4-10
1,5-10
2,6-9
3,7-8
4,5-10
...,...
951,3-4
952,1-6
953,2-5
954,5-2


In [41]:
def clean_ht_result(ht):
    """
    clean Result column and return df join back to original df
    """
    
    r_special = ht.Result.str.split(expand = True)[0]
    
    r_new_result = r_special.str.split('-', expand = True)
    r_new_result.rename(columns = {0:'win',1:'loss',2:'absent'}, inplace = True)
    
    return r_new_result

clean_ht_result(ht_new)

,win,loss,absent
0,1,4,10
1,5,10,None
2,6,9,None
3,7,8,None
4,5,10,None
...,...,...,...
951,3,4,None
952,1,6,None
953,2,5,None
954,5,2,None


# join ht and bz
# And
# add "next banzuke rank" to new df

In [43]:
ht_clean = clean_ht_result(ht_new)
ht_up = ht_new.join(ht_clean)
ht_up.head()

,Result,name,Rank,year,month,win,loss,absent
0,1-4-10,Hakuho,Y,2020,1,1,4,10
1,5-10,Takakeisho,O,2020,1,5,10,None
2,6-9 ↓,Asanoyama,S,2020,1,6,9,None
3,7-8 ↓,Abi,K,2020,1,7,8,None
4,5-10,Endo,M1,2020,1,5,10,None


In [44]:
idx = ['name','year','month']
nf = bz.join(ht_up.set_index(idx), on = idx)
nf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rank       350 non-null    object 
 1   name       350 non-null    object 
 2   pob        350 non-null    object 
 3   stable     350 non-null    object 
 4   birthdate  350 non-null    object 
 5   height     350 non-null    float64
 6   weight     350 non-null    float64
 7   year       350 non-null    int64  
 8   month      350 non-null    int64  
 9   Result     344 non-null    object 
 10  Rank       350 non-null    object 
 11  win        344 non-null    object 
 12  loss       344 non-null    object 
 13  absent     34 non-null     object 
dtypes: float64(2), int64(2), object(10)
memory usage: 38.4+ KB


In [45]:
nf.head()

,rank,name,pob,stable,birthdate,height,weight,year,month,Result,Rank,win,loss,absent
0,Y1e,Hakuho,Mongolia,Miyagino,11.03.1985,193.0,150.7,2020,1,1-4-10,Y,1,4,10
1,Y1w,Kakuryu,Mongolia,Michinoku,10.08.1985,186.0,150.0,2020,1,1-3-11,Y,1,3,11
2,O1e,Takakeisho,Hyogo,Chiganoura,05.08.1996,173.0,149.0,2020,1,5-10,O,5,10,None
3,O1w,Goeido,Osaka,Sakaigawa,06.04.1986,183.0,158.2,2020,1,11-4,O,11,4,None
4,S1e,Asanoyama,Toyama,Takasago,01.03.1994,189.0,158.0,2020,1,6-9 ↓,S,6,9,None


In [52]:
grouper = ['name']
sf = nf.sort_values(['name','year','month'])
sf['next_rank'] = sf.groupby(grouper)['rank'].shift(-1)
sf.head(20)

,rank,name,pob,stable,birthdate,height,weight,year,month,Result,Rank,win,loss,absent,next_rank
6,K1e,Abi,Saitama,Shikoroyama,04.05.1994,185.0,121.0,2020,1,7-8 ↓,K,7,8,None,M4w
84,M4w,Abi,Saitama,Shikoroyama,04.05.1994,185.0,121.0,2020,3,6-9,M4,6,9,None,M5e
156,M5e,Abi,Saitama,Shikoroyama,04.05.1994,185.0,121.0,2020,7,9-6,M5,9,6,None,M14w
246,M14w,Abi,Saitama,Shikoroyama,04.05.1994,185.0,121.0,2020,9,11-4 K,M14,11,4,None,J11w
343,J11w,Abi,Saitama,Shikoroyama,04.05.1994,185.0,121.0,2020,11,8-7,J11,8,7,None,NaN
139,J14w,Akiseyama,Aichi,Kise,18.07.1985,183.0,172.6,2020,3,9-6,J14,9,6,None,J9e
198,J9e,Akiseyama,Aichi,Kise,18.07.1985,183.0,172.6,2020,7,8-7,J9,8,7,None,J10e
270,J10e,Akiseyama,Aichi,Kise,18.07.1985,183.0,172.6,2020,9,11-4,J10,11,4,None,J1e
322,J1e,Akiseyama,Aichi,Kise,18.07.1985,183.0,172.6,2020,11,7-8,J1,7,8,None,NaN
59,J9w,Akua,Ibaraki,Tatsunami,06.11.1990,181.0,162.6,2020,1,10-5,J9,10,5,None,J6w


# Save results

In [53]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 6 to 311
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rank       350 non-null    object 
 1   name       350 non-null    object 
 2   pob        350 non-null    object 
 3   stable     350 non-null    object 
 4   birthdate  350 non-null    object 
 5   height     350 non-null    float64
 6   weight     350 non-null    float64
 7   year       350 non-null    int64  
 8   month      350 non-null    int64  
 9   Result     344 non-null    object 
 10  Rank       350 non-null    object 
 11  win        344 non-null    object 
 12  loss       344 non-null    object 
 13  absent     34 non-null     object 
 14  next_rank  266 non-null    object 
dtypes: float64(2), int64(2), object(11)
memory usage: 43.8+ KB


In [54]:
cols_to_save = ['rank', 'name', 'pob', 'birthdate', 'height', 'weight', 'year', 'month', 'win', 'loss', 'absent', 'next_rank']
sf[cols_to_save].to_csv('processed_ranks.csv')